Load moduls

In [187]:
from math import sin, cos, sqrt, atan2, radians
import re, string
import numpy as np
import scipy.stats as stats
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import warnings; warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format ='retina'

# House Data Cleaning

In [188]:
df = pd.read_csv('data/houses.csv')

In [189]:
df = df[~df.location.isnull()]
def calcdist(value):
    
    slt = value.split('\'')

    lat1 = radians(-37.810817)
    lon1 = radians(144.963135)
    lat2 = radians(float(slt[1]))
    lon2 = radians(float(slt[3]))

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    R = 6373.0
    distance = R * c

    return distance

df['dist2cbd'] = df.location.map(calcdist)

In [191]:
df.suburb = df.suburb.str.lower()

In [192]:
df.shape

(203198, 13)

In [195]:
df.dropna(inplace=True)
df.isnull().sum()

solddate       0
price          0
address1       0
postcode       0
latitude       0
longitude      0
bedrooms       0
bathrooms      0
carparkings    0
agentbrand     0
location       0
suburb         0
dist2cbd       0
dtype: int64

# EDA

In [8]:
choose_sub = ['yarraville', 'newport', 'spotswood', 'kensington',
              'ascot vale', 'moonee ponds', 'coburg', 'clifton hill',
              'northcote', 'malvern east', 'glen iris', 'elsternwick',
              'ormond']

def ch(x):
    if x in choose_sub:
        return True
    else:
        return False
    
ch = df[df.suburb.map(ch)]

In [9]:
ch.suburb.unique()

array(['yarraville', 'newport', 'spotswood', 'kensington', 'ascot vale',
       'moonee ponds', 'coburg', 'clifton hill', 'northcote',
       'malvern east', 'glen iris', 'elsternwick', 'ormond'], dtype=object)

In [10]:
c = ch.drop(['postcode', 'latitude', 'longitude', 'location'], axis=1)

# pd.to_datetime(c.solddate)
def qw(x):
    try:
        s = x.split('-')
        month = s[1]
        return True
    except:
        return False
c = c[c.solddate.map(qw)]

c.solddate = pd.to_datetime(c.solddate)
c.solddate = c.solddate.map(lambda x: x.strftime('%Y-%m'))

In [11]:
# c.groupby(by='solddate').count()['price'][56:-1].plot(kind='bar', figsize=(20,10))

In [12]:
# outliers 
c = c[~(c.price > 4000000)]

In [13]:
me = c.price.median()
ma = c.price.mean()
plt.figure(figsize=(18,10))

font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16,
        }
# plt.text(2500000, 0.00000125, s='Median '+str(me), fontsize=30, fontdict={'color':'red'})
# plt.text(2500000, 0.00000105, s='Mean  '+str(round(ma)), fontsize=30, fontdict={'color':'blue'})

# plt.plot([me, me], [0, 0.00000165], 'r-', lw=3)
# plt.plot([ma, ma], [0, 0.00000165], 'b-', lw=3)
# sns.distplot(c.price)

In [14]:
# c.groupby(by='agentbrand').count().sort_values(by='price', ascending=False)

In [15]:
c.bathrooms = c.bathrooms.astype(int)
c.carparkings = c.carparkings.astype(int)
corr = c[['bedrooms', 'bathrooms', 'carparkings', 'dist2cbd', 'price']].corr()
# sns.heatmap(corr, annot=True)

# Model

In [208]:
data = df.copy()
data.drop(['latitude', 'longitude', 'location', 'postcode'], axis=1, inplace=True)
data.drop(19517, axis=0, inplace=True)

In [209]:
def getStreetName(x):
    result = ''.join([i for i in x if not i.isdigit()])
    result = re.sub(r'[^\w]', ' ', result)
    result = ' '.join([x for x in result.split() if len(x) != 1])
    return result.lower()

data.address1 = data.address1.map(getStreetName)

In [210]:
data.head()

,solddate,price,address1,bedrooms,bathrooms,carparkings,agentbrand,suburb,dist2cbd
0,25-Aug-17,380000,katherine place,2.0,1,0,City Residential Real Estate,melbourne,1.201459
1,28-Aug-17,347000,franklin street,2.0,1,0,Caine Real Estate,melbourne,0.384706
2,26-Aug-17,458000,beckett street,2.0,1,0,MICM Real Estate,melbourne,0.267443
3,24-Aug-17,440000,therry street,2.0,1,0,Melcorp Property Pty Ltd,melbourne,0.467489
4,18-Aug-17,550000,collins street,2.0,2,0,MICM Real Estate,melbourne,1.085536


In [211]:
suburb = pd.read_csv('suburb.csv')
suburb.columns = ['suburb', 'post', 'region']
suburb.suburb = suburb.suburb.str.lower()
suburb.suburb = suburb.suburb.str.strip()

In [213]:
data2 = pd.merge(data, suburb, on='suburb', how='left')
data2.drop('post', axis=1, inplace=True)
data2.drop_duplicates(inplace=True)
data2.dropna(inplace=True)

In [215]:
data2

,solddate,price,address1,bedrooms,bathrooms,carparkings,agentbrand,suburb,dist2cbd,region
0,25-Aug-17,380000,katherine place,2.0,1,0,City Residential Real Estate,melbourne,1.201459,Inner
3,28-Aug-17,347000,franklin street,2.0,1,0,Caine Real Estate,melbourne,0.384706,Inner
6,26-Aug-17,458000,beckett street,2.0,1,0,MICM Real Estate,melbourne,0.267443,Inner
9,24-Aug-17,440000,therry street,2.0,1,0,Melcorp Property Pty Ltd,melbourne,0.467489,Inner
12,18-Aug-17,550000,collins street,2.0,2,0,MICM Real Estate,melbourne,1.085536,Inner
15,16-Aug-17,564700,beckett street,2.0,1,0,LJ Hooker City Residential,melbourne,0.323293,Inner
18,15-Aug-17,623000,swanston street,2.0,2,0,Nelson Alexander Docklands,melbourne,0.529941,Inner
21,07-Aug-17,555000,russell street,2.0,1,0,Harcourts Melbourne City,melbourne,0.602914,Inner
24,29-Jul-17,565000,franklin street,2.0,1,0,MICM Real Estate,melbourne,0.335164,Inner
27,20-Jul-17,379000,beckett street,2.0,1,0,Greg Hocking City Residential,melbourne,0.216394,Inner
